In [1]:
import re
import time
from appium import webdriver
from appium.options.android import UiAutomator2Options
from appium.webdriver.common.appiumby import AppiumBy
from mock_click import find_and_click_element

NETWORK_TYPE_MAPPING = {
    "5G": ["5G NSA", "5G SA"],
    "4G": ["4G"],
    "3G": ["3G"],
    "2G": ["2G"]
}


def wake_up_screen(driver):
    try:
        driver.press_keycode(224)
        time.sleep(0.3)
    except Exception as e:
        print(f"唤醒屏幕失败: {e}")

def enter_data_setting(driver):
    wake_up_screen(driver)
    go_home(driver)
    try:
        find_and_click_element(driver, "Settings")
        time.sleep(0.2)
    except Exception as e:
        print('Error: settings icon not found')
        return
    try:
        find_and_click_element(driver, "Mobile networks")
        time.sleep(0.2)
    except Exception as e:
        print('Error: Mobile networks icon not found')

def enter_prefer_network(driver):
    try:
        find_and_click_element(driver, "Preferred network type")
        time.sleep(0.2)
    except Exception as e:
        print('Error: Preferred network type icon not found')

def go_home(driver):
    try:
        driver.press_keycode(3)
        time.sleep(0.1)
    except Exception as e:
        print(f"返回主屏幕失败: {e}")

def open_sim_network_settings(driver, sim_label):
    wake_up_screen(driver)
    enter_data_setting(driver)
    time.sleep(0.3)
    try:
        driver.find_element(AppiumBy.ID, sim_label).click()
        time.sleep(0.2)
    except Exception as e:
        print(f"Error in open_sim_network_settings: {e}")

def select_preferred_network_type(driver, target_keyword):
    synonyms = {
        "4G": ["4G", "LTE"],
        "LTE": ["4G", "LTE"],
        "5G": ["5G"]
    }
    try:
        elements = driver.find_elements(AppiumBy.XPATH, "//*[@text]")
        for element in elements:
            text = element.text
            if text:
                for synonym in synonyms.get(target_keyword.upper(), [target_keyword]):
                    if synonym in text:
                        element.click()
                        print(f"Successfully selected network type: {text}")
                        return
        print(f"Error: Element with text containing '{target_keyword}' or its synonyms not found.")
    except Exception as e:
        print(f"Error selecting network type containing '{target_keyword}': {e}")

def validate_network_registration(driver, sim_slot, target_networks):
    try:
        result = driver.execute_script("mobile: shell", {
            "command": "dumpsys",
            "args": ["telephony.registry"],
            "includeStderr": True,
            "timeout": 5000
        })

        if "stdout" not in result:
            print("Error: Failed to fetch telephony registry information.")
            return False

        output = result["stdout"]
        phone_id = 0 if sim_slot.upper() == "SIM1" else 1
        sim_identifier = f"Phone Id={phone_id}"

        if sim_identifier not in output:
            print(f"Error: SIM slot {sim_slot} (Phone Id={phone_id}) not found in telephony registry output.")
            return False

        sim_info_start = output.find(sim_identifier)
        if sim_info_start == -1:
            print(f"Error: SIM slot {sim_slot} (Phone Id={phone_id}) not found in telephony registry output.")
            return False
        next_sim_start = output.find("Phone Id=", sim_info_start + len(sim_identifier))
        sim_info_final = output.find("local logs")  # 假设 "local logs" 是全局结束标志
        sim_info_end = min(filter(lambda x: x != -1, [next_sim_start, sim_info_final]))
        sim_info = output[sim_info_start:sim_info_end]
        # 初始化提取结果
        voice_radio_tech = None
        data_radio_tech = None
        display_network = None
        override_network = None
        data_connection_state = None

        # 提取具体字段
        for line in sim_info.splitlines():
            if "getRilVoiceRadioTechnology" in line:
                match = re.search(r"getRilVoiceRadioTechnology=(\d+)", line)
                if match:
                    voice_radio_tech = match.group(1).strip()
                # print(f"Extracted voice_radio_tech: {voice_radio_tech}")
            if "getRilDataRadioTechnology" in line:
                match = re.search(r"getRilDataRadioTechnology=(\d+)", line)
                if match:
                    data_radio_tech = match.group(1).strip()
                # print(f"Extracted data_radio_tech: {data_radio_tech}")
            if "mTelephonyDisplayInfo" in line:
                if "network=" in line:
                    display_network = line.split("network=")[-1].split(",")[0].strip()
                if "overrideNetwork=" in line:
                    override_network = line.split("overrideNetwork=")[-1].split(",")[0].strip()
            if "mDataConnectionState" in line:
                match = re.search(r"mDataConnectionState=(\d+)", line)
                if match:
                    data_connection_state = int(match.group(1))
            # print(f"SIM{phone_id + 1} Info: voice_radio_tech={voice_radio_tech}, data_radio_tech={data_radio_tech}, display_network={display_network}, override_network={override_network}")
        # 根据 mDataConnectionState 跳过某些验证逻辑
        if data_connection_state == 0:  # 无数据连接
            print(f"SIM{phone_id + 1}: Data connection is inactive, skipping data_radio_tech validation.")
            for target_network in target_networks:
                if target_network.lower() == "3g" and voice_radio_tech == "3":
                    print(f"Validation successful: SIM{phone_id + 1} is registered on 3G network (voice only).")
                    return True
                elif target_network.lower() == "2g" and voice_radio_tech == "16":
                    print(f"Validation successful: SIM{phone_id + 1} is registered on 2G network (voice only).")
                    return True
        else:  # 数据连接活动
        # 验证目标网络类型
            for target_network in target_networks:
                if target_network.lower() == "5g sa" and voice_radio_tech == "20" and data_radio_tech == "20":
                    print(f"Validation successful: SIM{phone_id + 1} is registered on 5G SA network.")
                    return True
                elif target_network.lower() == "5g nsa" and voice_radio_tech == "14" and override_network == "NR_NSA":
                    print(f"Validation successful: SIM{phone_id + 1} is registered on 5G NSA network.")
                    return True
                elif target_network.lower() == "4g" and voice_radio_tech == "14" and display_network in ["LTE", "LTE_CA"]:
                    print(f"Validation successful: SIM{phone_id + 1} is registered on 4G network.")
                    return True
                elif target_network.lower() == "3g" and voice_radio_tech == "3":
                    print(f"Validation successful: SIM{phone_id + 1} is registered on 3G network.")
                    return True
                elif target_network.lower() == "2g" and voice_radio_tech == "16" and data_radio_tech == "2":
                    print(f"Validation successful: SIM{phone_id + 1} is registered on 2G network.")
                    return True

        return False
        print(f"{sim_slot} is not registered on {target_networks}.")
    except Exception as e:
        print(f"Error validating network registration: {e}")
        return False

def ensure_network_registration(driver, sim_slot, target_network, retries=5, interval=3):
    target_networks = NETWORK_TYPE_MAPPING.get(target_network.upper(), [target_network])
    for attempt in range(retries):
        # print(f"验证 {sim_slot} 是否注册到 {target_networks}，第 {attempt + 1} 次...")
        if validate_network_registration(driver, sim_slot, target_networks):
            return True
        time.sleep(interval)
    print(f"{sim_slot} 未能注册到目标网络 {target_networks}")
    return False

def switch_network_type(driver, sim_slot, target_network):
    wake_up_screen(driver)
    target_sim_id = "com.android.phone:id/sim_1" if sim_slot.upper() == "SIM1" else "com.android.phone:id/sim_2"
    open_sim_network_settings(driver, target_sim_id)
    enter_prefer_network(driver)
    select_preferred_network_type(driver, target_network)

def switch_and_validate_networks(driver, sim_slots, target_networks):
    for sim_slot, network in zip(sim_slots, target_networks):
        switch_network_type(driver, sim_slot, network)
    results = {}
    for sim_slot, network in zip(sim_slots, target_networks):
        results[sim_slot] = ensure_network_registration(driver, sim_slot, network)
    return results

def __main__(driver, sim_slots, target_networks):
    results = switch_and_validate_networks(driver, sim_slots, target_networks)
    for sim_slot, success in results.items():
        if success:
            print(f"{sim_slot} 成功注册到目标网络。")
        else:
            print(f"{sim_slot} 未能注册到目标网络。")


In [2]:
options = UiAutomator2Options()
options.platform_name = "Android"
options.device_name = "DUT"

In [3]:
try:
    # 启动 Appium 会话
    driver = webdriver.Remote("http://127.0.0.1:4723", options=options)
        # 测试主函数
    validate_network_registration(driver, "SIM2", "2G")
    ensure_network_registration(driver, "SIM2", "2G")
        # 关闭会话
    driver.quit()
    # print("代码运行正常！")

except Exception as e:
    print(f"测试运行失败: {e}")

SIM2: Data connection is inactive, skipping data_radio_tech validation.
SIM2: Data connection is inactive, skipping data_radio_tech validation.
Validation successful: SIM2 is registered on 2G network (voice only).
